In [28]:
import re
from json import dump
import requests

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 51) # update this to your liking

# begin code
url_links = []
property_metadata = defaultdict(dict)

# generate list of urls to visit
for page in N_PAGES:
    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=price-desc&page={page}"
    bs_object = BeautifulSoup(requests.get(
    url, headers=headers).text, "html.parser")

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object \
        .find(
            "ul",
            {"data-testid": "results"}
        ) \
        .findAll(
            "a",
            href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
        )

    for link in index_links:
        # if its a property address, add it to the list
        if 'address' in link['class']:
            url_links.append(link['href'])


# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    # bs_object = BeautifulSoup(urlopen(property_url), "lxml")
    bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object \
        .find("h1", {"class": "css-164r41r"}) \
        .text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object \
        .find("div", {"data-testid": "listing-details__summary-title"}) \
        .text

    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object \
                .find(
                    "a",
                    {"target": "_blank", 'rel': "noopener noreferer"}
                ) \
                .attrs['href']
        )[0].split(',')
    ]

    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    # property_metadata[property_url]['external_information'] = bs_object \
    # .find("div", {"data-testid": "strip-content-list"}).text
    # print("yes")


    property_metadata[property_url]['desc'] = re \
        .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
        .strip('</p>')

# output to example json in data/raw/
with open('../data/raw/example.json', 'w') as f:
    dump(property_metadata, f)

In [29]:
import json
import pandas as pd
  
# Opening JSON file
f = open('../data/raw/example.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
df2 = pd.DataFrame.from_dict(data, orient="index")

In [30]:
df2.shape

(999, 5)

In [31]:
l = []
for i in range(len(df2)):
    if(len(df2['rooms'][i]) != 0):
        l.append(True)
    else:
        l.append(False)
df3 = df2[l]

In [32]:
df3 = df3.reset_index()
df3.rename(columns={'index':'link'}, inplace = True)

In [33]:
df3

,link,name,cost_text,coordinates,rooms,desc
0,https://www.domain.com.au/upstairs-2c-staley-s...,Upstairs 2C Staley Street Brunswick VIC 3056,"$35,000 Annually","[-37.7655919, 144.9633048]","[[0 Beds], [0 Baths], [2 Parking]]",1st floor offices/studios\nEasy walking distan...
1,https://www.domain.com.au/3502-14-16-the-espla...,3502/14-16 The Esplanade St Kilda VIC 3182,"$6,000","[-37.8650177, 144.9746821]","[[3 Beds], [3 Baths], [3 Parking]]",Inspired by the interplay of timeless design a...
2,https://www.domain.com.au/9-lansdowne-street-b...,9 Lansdowne Street Blairgowrie VIC 3942,"$5,000 per week","[-38.372703, 144.7856897]","[[3 Beds], [2 Baths], [3 Parking], []]",Phone enquiry code for this property : 2751
3,https://www.domain.com.au/52-black-st-brighton...,52 Black St Brighton VIC 3186,From $5500 per week,"[-37.9159452, 144.9989003]","[[3 Beds], [4 Baths], [3 Parking], []]","class=""css-dxogle"">* Unverified feature<svg a..."
4,https://www.domain.com.au/4203-35-spring-stree...,4203/35 Spring Street Melbourne VIC 3000,$4500 Per Week,"[-37.8141725, 144.9740049]","[[3 Beds], [2 Baths], [4 Parking]]",Possibly the most prestigious CBD address in t...
...,...,...,...,...,...,...
989,https://www.domain.com.au/1-bluff-rd-merricks-...,1 Bluff Rd Merricks Beach VIC 3926,$795,"[-38.3997481, 145.1048468]","[[3 Beds], [2 Baths], [2 Parking]]","class=""css-dxogle"">* Unverified feature<svg a..."
990,https://www.domain.com.au/1-11-bay-street-park...,1/11 Bay Street Parkdale VIC 3195,$795,"[-38.00308, 145.0810804]","[[3 Beds], [2 Baths], [2 Parking]]","class=""css-dxogle"">* Unverified feature<svg a..."
991,https://www.domain.com.au/2-sutton-grove-richm...,2 Sutton Grove Richmond VIC 3121,$795 per week,"[-37.8225969, 144.990798]","[[2 Beds], [2 Baths], [1 Parking]]",Angela Clark
992,https://www.domain.com.au/2-261-williams-road-...,2/261 Williams Road South Yarra VIC 3141,$795 per week,"[-37.8420314, 145.004217]","[[2 Beds], [1 Bath], [1 Parking]]","Like nothing you've ever encountered, this tho..."


In [34]:
import re
bedroom_list = []
bathroom_list = []
parking_list = []
for i in range(len(df3)):
    bedroom_list.append(re.search('\d+', df3['rooms'][i][0][0]).group())
    bathroom_list.append(re.search('\d+', df3['rooms'][i][1][0]).group())
    if(len(df3['rooms'][i][2])!= 0):
        parking_list.append(re.search('\d+', df3['rooms'][i][2][0]).group())
    else:
        parking_list.append('0')
    # df3['bathroom'][i] = re.search('\d+', df3['rooms'][i][1][0]).group()
    # df3['parking'][i] = re.search('\d+', df3['rooms'][i][2][0]).group()

In [35]:
df3['bedroom'] = bedroom_list
df3['bathroom'] = bathroom_list
df3['parking'] = parking_list

In [36]:
df3

,link,name,cost_text,coordinates,rooms,desc,bedroom,bathroom,parking
0,https://www.domain.com.au/upstairs-2c-staley-s...,Upstairs 2C Staley Street Brunswick VIC 3056,"$35,000 Annually","[-37.7655919, 144.9633048]","[[0 Beds], [0 Baths], [2 Parking]]",1st floor offices/studios\nEasy walking distan...,0,0,2
1,https://www.domain.com.au/3502-14-16-the-espla...,3502/14-16 The Esplanade St Kilda VIC 3182,"$6,000","[-37.8650177, 144.9746821]","[[3 Beds], [3 Baths], [3 Parking]]",Inspired by the interplay of timeless design a...,3,3,3
2,https://www.domain.com.au/9-lansdowne-street-b...,9 Lansdowne Street Blairgowrie VIC 3942,"$5,000 per week","[-38.372703, 144.7856897]","[[3 Beds], [2 Baths], [3 Parking], []]",Phone enquiry code for this property : 2751,3,2,3
3,https://www.domain.com.au/52-black-st-brighton...,52 Black St Brighton VIC 3186,From $5500 per week,"[-37.9159452, 144.9989003]","[[3 Beds], [4 Baths], [3 Parking], []]","class=""css-dxogle"">* Unverified feature<svg a...",3,4,3
4,https://www.domain.com.au/4203-35-spring-stree...,4203/35 Spring Street Melbourne VIC 3000,$4500 Per Week,"[-37.8141725, 144.9740049]","[[3 Beds], [2 Baths], [4 Parking]]",Possibly the most prestigious CBD address in t...,3,2,4
...,...,...,...,...,...,...,...,...,...
989,https://www.domain.com.au/1-bluff-rd-merricks-...,1 Bluff Rd Merricks Beach VIC 3926,$795,"[-38.3997481, 145.1048468]","[[3 Beds], [2 Baths], [2 Parking]]","class=""css-dxogle"">* Unverified feature<svg a...",3,2,2
990,https://www.domain.com.au/1-11-bay-street-park...,1/11 Bay Street Parkdale VIC 3195,$795,"[-38.00308, 145.0810804]","[[3 Beds], [2 Baths], [2 Parking]]","class=""css-dxogle"">* Unverified feature<svg a...",3,2,2
991,https://www.domain.com.au/2-sutton-grove-richm...,2 Sutton Grove Richmond VIC 3121,$795 per week,"[-37.8225969, 144.990798]","[[2 Beds], [2 Baths], [1 Parking]]",Angela Clark,2,2,1
992,https://www.domain.com.au/2-261-williams-road-...,2/261 Williams Road South Yarra VIC 3141,$795 per week,"[-37.8420314, 145.004217]","[[2 Beds], [1 Bath], [1 Parking]]","Like nothing you've ever encountered, this tho...",2,1,1


In [37]:
df4 = df3.iloc[:,[1,2,3,6,7,8]]
df4.to_csv('../data/curated/initial.csv')